In [1]:
import getpass
import os


def _set_if_undefined(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"Please provide your {var}")


_set_if_undefined("GEMINI_API_KEY")
#_set_if_undefined("TAVILY_API_KEY")

In [2]:
from typing import Annotated

#from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.tools import tool
from langchain_experimental.utilities import PythonREPL




#tavily_tool = TavilySearchResults(max_results=5)

# Warning: This executes code locally, which can be unsafe when not sandboxed

repl = PythonREPL()


@tool
def python_repl_tool(
    code: Annotated[str, "The python code to execute to generate your chart."],
):
    """Use this to execute python code. If you want to see the output of a value,
    you should print it out with `print(...)`. This is visible to the user."""
    try:
        result = repl.run(code)
    except BaseException as e:
        return f"Failed to execute. Error: {repr(e)}"
    result_str = f"Successfully executed:\n```python\n{code}\n```\nStdout: {result}"
    return (
        result_str + "\n\nIf you have completed all tasks, respond with FINAL ANSWER."
    )

In [40]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.prebuilt import create_react_agent
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", api_key=os.getenv('GEMINI_API_KEY'))

In [52]:
import sys
sys.path.append(os.path.abspath('../../..'))

from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
from langchain_mcp_adapters.tools import load_mcp_tools
from langchain_mcp_adapters.client import MultiServerMCPClient


In [4]:
from typing import Literal
from typing_extensions import TypedDict

from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.graph import MessagesState, END
from langgraph.types import Command


members = ["news_analyst", "coder"]
# Our team supervisor is an LLM node. It just picks the next agent to process
# and decides when the work is completed
options = members + ["FINISH"]

system_prompt = (
    "You are a supervisor tasked with managing a conversation between the"
    f" following workers: {members}. Given the following user request,"
    " respond with the worker to act next. Each worker will perform a"
    " task and respond with their results and status. When finished,"
    " respond with FINISH."
)


class Router(TypedDict):
    """Worker to route to next. If no workers needed, route to FINISH."""

    next: Literal[*options]





class State(MessagesState):
    next: str


def supervisor_node(state: State) -> Command[Literal[*members, "__end__"]]:
    messages = [
        {"role": "system", "content": system_prompt},
    ] + state["messages"]
    response = llm.with_structured_output(Router).invoke(messages)
    goto = response["next"]
    if goto == "FINISH":
        goto = END

    return Command(goto=goto, update={"next": goto})

In [58]:
from langchain_core.messages import HumanMessage
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import create_react_agent


async def make_news_agent():     
    async with MultiServerMCPClient(
        {
            "tickertick": {
                "command": "python",
                # Make sure to update to the full absolute path to your math_server.py file
                "args": ["/Users/chen/Library/Mobile Documents/com~apple~CloudDocs/NYU/SPRING 25/TECH-UB 24/StocksFlags/src/mcp_server/tickertick.py"],
                "transport": "stdio",
            },
        }
    ) as client:
        tickertick = client.get_tools()
        agent = create_react_agent(llm, tools=tickertick, prompt="You are a news analyst. You get the news based on user's request and return the summary of the news.")
    return agent

news_analyst_agent = await make_news_agent()

async def news_analyst_node(state: State) -> Command[Literal["supervisor"]]:
    result = await news_analyst_agent.ainvoke(state)
    return Command(
        update={
            "messages": [
                HumanMessage(content=result["messages"][-1].content, name="news_analyst")
            ]
        },
        goto="supervisor",
    )





# NOTE: THIS PERFORMS ARBITRARY CODE EXECUTION, WHICH CAN BE UNSAFE WHEN NOT SANDBOXED
code_agent = create_react_agent(llm, tools=[python_repl_tool])


def code_node(state: State) -> Command[Literal["supervisor"]]:
    result = code_agent.invoke(state)
    return Command(
        update={
            "messages": [
                HumanMessage(content=result["messages"][-1].content, name="coder")
            ]
        },
        goto="supervisor",
    )


builder = StateGraph(State)
builder.add_edge(START, "supervisor")
builder.add_node("supervisor", supervisor_node)
builder.add_node("news_analyst", news_analyst_node)
builder.add_node("coder", code_node)
graph = builder.compile()

I0000 00:00:1744396976.599783 16200342 fork_posix.cc:75] Other threads are currently calling into gRPC, skipping fork() handlers


In [61]:
news_analyst_agent = await make_news_agent()
query = "What's some news about nvda?"
response = await news_analyst_agent.ainvoke({"messages": query})

I0000 00:00:1744397191.187657 16200342 fork_posix.cc:75] Other threads are currently calling into gRPC, skipping fork() handlers


In [62]:
print(response)

{'messages': [HumanMessage(content="What's some news about nvda?", additional_kwargs={}, response_metadata={}, id='0a0ef151-bd86-49a1-bda9-a93a375e05c9'), AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_ticker_news_tool', 'arguments': '{"ticker": "NVDA"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run-36b7ada9-3b38-4742-8e2e-44d146fb1c5f-0', tool_calls=[{'name': 'get_ticker_news_tool', 'args': {'ticker': 'NVDA'}, 'id': 'a4da0c54-d890-45ac-bd94-c82af643997c', 'type': 'tool_call'}], usage_metadata={'input_tokens': 661, 'output_tokens': 10, 'total_tokens': 671, 'input_token_details': {'cache_read': 0}}), ToolMessage(content='Error: ClosedResourceError()\n Please fix your mistakes.', name='get_ticker_news_tool', id='0e21d798-3326-4ad9-a2c5-ee52526bcb81', tool_call_id='a4da0c54-d890-45ac-bd94-c82af643997c', status='error'), AIMessage(content=

In [59]:
async for s in graph.astream(
    {"messages": [("user", "What's some news about nvda?")]}, subgraphs=True
):
    print(s)
    print("----")

/Users/chen/anaconda3/envs/stocksflags/lib/python3.11/site-packages/langchain_google_genai/chat_models.py:1410: UserWarning: ChatGoogleGenerativeAI.with_structured_output with dict schema has changed recently to align with behavior of other LangChain chat models. More context: https://github.com/langchain-ai/langchain-google/pull/772
  warnings.warn(


((), {'supervisor': {'next': 'news_analyst'}})
----
(('news_analyst:b9e553c5-8548-c12c-10ab-e34a2240d102',), {'agent': {'messages': [AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_ticker_news_tool', 'arguments': '{"ticker": "NVDA"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run-9309ae9d-63b2-4d9a-924d-471bd9e1f269-0', tool_calls=[{'name': 'get_ticker_news_tool', 'args': {'ticker': 'NVDA'}, 'id': 'ff28a9d1-41e3-49d0-8528-58af4b15d9f1', 'type': 'tool_call'}], usage_metadata={'input_tokens': 661, 'output_tokens': 10, 'total_tokens': 671, 'input_token_details': {'cache_read': 0}})]}})
----
(('news_analyst:b9e553c5-8548-c12c-10ab-e34a2240d102',), {'tools': {'messages': [ToolMessage(content='Error: ClosedResourceError()\n Please fix your mistakes.', name='get_ticker_news_tool', id='5c865661-2109-4fc1-b9ec-89fdd21a4155', tool_call_id='ff28a9d

/Users/chen/anaconda3/envs/stocksflags/lib/python3.11/site-packages/langchain_google_genai/chat_models.py:1410: UserWarning: ChatGoogleGenerativeAI.with_structured_output with dict schema has changed recently to align with behavior of other LangChain chat models. More context: https://github.com/langchain-ai/langchain-google/pull/772
  warnings.warn(


((), {'supervisor': {'next': 'news_analyst'}})
----
(('news_analyst:673a2636-61ad-6611-9ca9-9e6b0dcf13d5',), {'agent': {'messages': [AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_ticker_news_tool', 'arguments': '{"ticker": "NVDA"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run-8208d490-3221-421b-bb5d-6044da10fd12-0', tool_calls=[{'name': 'get_ticker_news_tool', 'args': {'ticker': 'NVDA'}, 'id': '95cf4f30-5bd2-42b1-a0a5-e09f03131747', 'type': 'tool_call'}], usage_metadata={'input_tokens': 685, 'output_tokens': 10, 'total_tokens': 695, 'input_token_details': {'cache_read': 0}})]}})
----
(('news_analyst:673a2636-61ad-6611-9ca9-9e6b0dcf13d5',), {'tools': {'messages': [ToolMessage(content='Error: ClosedResourceError()\n Please fix your mistakes.', name='get_ticker_news_tool', id='3523c6ce-f927-430f-bf19-12cea2e3a6b7', tool_call_id='95cf4f3

/Users/chen/anaconda3/envs/stocksflags/lib/python3.11/site-packages/langchain_google_genai/chat_models.py:1410: UserWarning: ChatGoogleGenerativeAI.with_structured_output with dict schema has changed recently to align with behavior of other LangChain chat models. More context: https://github.com/langchain-ai/langchain-google/pull/772
  warnings.warn(


...

/Users/chen/anaconda3/envs/stocksflags/lib/python3.11/site-packages/langchain_google_genai/chat_models.py:1410: UserWarning: ChatGoogleGenerativeAI.with_structured_output with dict schema has changed recently to align with behavior of other LangChain chat models. More context: https://github.com/langchain-ai/langchain-google/pull/772
  warnings.warn(


((), {'supervisor': {'next': 'news_analyst'}})
----
(('news_analyst:98114e51-28a0-31e1-c17e-2d67179d0165',), {'agent': {'messages': [AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_ticker_news_tool', 'arguments': '{"ticker": "NVDA"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run-08e13b89-96f2-455b-b3cd-dc6ed3dfc280-0', tool_calls=[{'name': 'get_ticker_news_tool', 'args': {'ticker': 'NVDA'}, 'id': '3936485c-afda-476c-8920-b7dcb53867f5', 'type': 'tool_call'}], usage_metadata={'input_tokens': 733, 'output_tokens': 10, 'total_tokens': 743, 'input_token_details': {'cache_read': 0}})]}})
----
(('news_analyst:98114e51-28a0-31e1-c17e-2d67179d0165',), {'tools': {'messages': [ToolMessage(content='Error: ClosedResourceError()\n Please fix your mistakes.', name='get_ticker_news_tool', id='42551c70-5882-4aec-8337-3b5e9d274560', tool_call_id='3936485

/Users/chen/anaconda3/envs/stocksflags/lib/python3.11/site-packages/langchain_google_genai/chat_models.py:1410: UserWarning: ChatGoogleGenerativeAI.with_structured_output with dict schema has changed recently to align with behavior of other LangChain chat models. More context: https://github.com/langchain-ai/langchain-google/pull/772
  warnings.warn(


((), {'supervisor': {'next': 'news_analyst'}})
----


CancelledError: 

for s in graph.stream(
    {
        "messages": [
            (
                "user",
                "Find the latest GDP of New York and California, then calculate the average",
            )
        ]
    },
    subgraphs=True,
):
    print(s)
    print("----")